<a href="https://colab.research.google.com/github/rani227/rag-based-recommendation/blob/main/shl_recommendation_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade -q openai

In [ ]:
!pip install faiss-cpu rapidfuzz python-dotenv rank-bm25 nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 47.7 MB/s eta 0:00:00


In [ ]:
%load_ext cudf.pandas
import pandas as pd

data = pd.read_csv('shl_dataset.csv')

In [ ]:
data["embedding_input"] = data.apply(
    lambda row: f"{row['name']}. {row['description']}. Type: {row['test_type']}. Duration: {row['duration_minutes']} mins.", axis=1
)

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize

In [ ]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
embeddings = embedding_model.encode(data["embedding_input"].tolist(), show_progress_bar=True)
normalized_embeddings = normalize(embeddings, axis=1, norm='l2')

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import faiss

In [ ]:
embedding_dim = normalized_embeddings.shape[1]
faiss_index = faiss.IndexFlatIP(embedding_dim)
faiss_index.add(normalized_embeddings)

In [ ]:
def retrieve_candidates(query_embedding, parsed_filters, top_k=20):
    # Step 5.1: Semantic Search via FAISS
    query_embedding = normalize(query_embedding, axis=1, norm='l2')
    scores, indices = faiss_index.search(query_embedding, top_k)

    # Step 5.2: Filter Results with Hard Constraints
    filtered_results = []
    for idx, score in zip(indices[0], scores[0]):
        row = data.iloc[idx]

        # Hard filters (optional, adjustable)
        if parsed_filters.get("duration_minutes"):
            max_duration = parsed_filters["duration_minutes"].get("max")
            if max_duration and row["duration_minutes"] > max_duration:
                continue

        if parsed_filters.get("adaptive_required") and not row["adaptive_support"]:
            continue

        if parsed_filters.get("remote_required") and not row["remote_support"]:
            continue

        filtered_results.append((row, score))

    return filtered_results

In [ ]:
import numpy as np

In [ ]:
def retrieve_and_debug(query: str, parsed_query: dict, top_k=10):
    query_embedding = embedding_model.encode([query])
    results = retrieve_candidates(query_embedding, parsed_query, top_k=top_k*2)  # Retrieve more, then rerank

    print(f"\n🔍 Query: {query}")
    print(f"📊 Parsed Filters: {parsed_query}\n")

    if not results:
        print("⚠️ No matching assessments found after filtering. Returning top FAISS result (unfiltered).\n")
        scores, indices = faiss_index.search(normalize(query_embedding, axis=1), 1)
        fallback = data.iloc[indices[0][0]]
        return pd.DataFrame([fallback])

    # Rank by semantic score (or later: composite score)
    sorted_results = sorted(results, key=lambda x: x[1], reverse=True)[:top_k]

    for i, (row, score) in enumerate(sorted_results):
        print(f"[{i+1}] {row['name']} - Score: {score:.4f} | Duration: {row['duration_minutes']} | Adaptive: {row['adaptive_support']} | Remote: {row['remote_support']}")

    return pd.DataFrame([r[0] for r in sorted_results])

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv(".env")
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found. Please upload your .env file.")

In [ ]:
from openai import OpenAI
import json
client = OpenAI(api_key=api_key)

def parse_query_with_llm(query: str) -> dict:
    few_shot_examples = """
Q: I'm looking for a Java test that assesses both technical and soft skills under 40 minutes.
A:
{
  "skills": ["java"],
  "soft_skills": ["communication", "teamwork"],
  "duration_minutes": { "max": 40 },
  "adaptive_required": false,
  "remote_required": false
}

Q: Need an adaptive test for Python developers that works well for remote hiring.
A:
{
  "skills": ["python"],
  "soft_skills": [],
  "duration_minutes": null,
  "adaptive_required": true,
  "remote_required": true
}

Q: """ + query + """
A:
"""

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # GPT-4 can be swapped in later
        messages=[
            {"role": "system", "content": "You extract structured filters from hiring assessment queries. Respond ONLY with JSON."},
            {"role": "user", "content": few_shot_examples}
        ],
        temperature=0,
    )

    # Capture raw text response
    content = response.choices[0].message.content.strip()

    try:
        parsed = json.loads(content)
        return parsed
    except json.JSONDecodeError:
        print("LLM response could not be parsed as JSON:")
        print(content)
        return {}

In [ ]:
query = "I need an assessment for advanced Java developers that also evaluates their ability to communicate and work in teams. Preferably under 30 minutes and adaptive."

parsed_query = parse_query_with_llm(query)
print("✅ Parsed Query:")
print(json.dumps(parsed_query, indent=2))

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}